In [1]:
import pandas as pd 
import numpy as np 
import cv2 # Used to manipulated the images 
np.random.seed(1337) # The seed I used - pick your own or comment out for a random seed. A constant seed allows for better comparisons though

# Import Keras 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam, Nadam, SGD, RMSprop

Using TensorFlow backend.


In [3]:
df_train = pd.read_json("C:\\Users\\dshah\\Desktop\\data\\processed\\train.json") # this is a dataframe

In [4]:
def get_scaled_imgs(df):
    imgs = []
    
    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)
        
        # Rescale
        a = (band_1 - band_1.mean()) / (band_1.max() - band_1.min())
        b = (band_2 - band_2.mean()) / (band_2.max() - band_2.min())
        c = (band_3 - band_3.mean()) / (band_3.max() - band_3.min())

        imgs.append(np.dstack((a, b, c)))

    return np.array(imgs)

In [5]:
Xtrain = get_scaled_imgs(df_train)

In [6]:
Ytrain = np.array(df_train['is_iceberg'])


In [7]:
df_train.inc_angle = df_train.inc_angle.replace('na',0)
idx_tr = np.where(df_train.inc_angle>0)

In [8]:
Ytrain = Ytrain[idx_tr[0]]
Xtrain = Xtrain[idx_tr[0],...]

In [9]:
def get_more_images(imgs):
    
    more_images = []
    vert_flip_imgs = []
    hori_flip_imgs = []
      
    for i in range(0,imgs.shape[0]):
        a=imgs[i,:,:,0]
        b=imgs[i,:,:,1]
        c=imgs[i,:,:,2]
        
        av=cv2.flip(a,1)
        ah=cv2.flip(a,0)
        bv=cv2.flip(b,1)
        bh=cv2.flip(b,0)
        cv=cv2.flip(c,1)
        ch=cv2.flip(c,0)
        
        vert_flip_imgs.append(np.dstack((av, bv, cv)))
        hori_flip_imgs.append(np.dstack((ah, bh, ch)))
      
    v = np.array(vert_flip_imgs)
    h = np.array(hori_flip_imgs)
       
    more_images = np.concatenate((imgs,v,h))
    
    return more_images

In [36]:
def get_more_images2(imgs):
    
    more_images = []
    vert_flip_imgs = []
    hori_flip_imgs = []
      
    for i in range(0,imgs.shape[0]):
        a=imgs[i,:,:,0]
        b=imgs[i,:,:,1]
        c=imgs[i,:,:,2]
        
        av3=cv2.rotate(a,1)
        ah3=cv2.rotate(a,0)
        bv3=cv2.rotate(b,1)
        bh3=cv2.rotate(b,0)
        cv3=cv2.rotate(c,1)
        ch3=cv2.rotate(c,0)
        
        vert_flip_imgs.append(np.dstack((av3, bv3, cv3)))
        hori_flip_imgs.append(np.dstack((ah3, bh3, ch3)))
      
    v = np.array(vert_flip_imgs)
    h = np.array(hori_flip_imgs)
       
    more_images = np.concatenate((imgs,v,h))
    
    return more_images

In [42]:
Xtr_more1 = get_more_images(Xtrain)

In [43]:
Xtr_more = get_more_images2(Xtr_more1)

In [93]:
Xtrain.shape

(1471, 75, 75, 3)

In [94]:
Ytrain.shape

(1471,)

In [57]:
Xtr_more1.shape

(4413, 75, 75, 3)

In [45]:
Xtr_more.shape

(13239, 75, 75, 3)

In [62]:
Ytr_more.shape

(13239,)

In [61]:
Ytr_more = np.concatenate((Ytrain,Ytrain,Ytrain,Ytrain,Ytrain,Ytrain,Ytrain,Ytrain,Ytrain))

In [63]:
def getModel():
    #Build keras model
    
    model=Sequential()
    
    # CNN 1
    model.add(Conv2D(64, kernel_size=(3, 3),activation='relu', input_shape=(75, 75, 3)))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.15))
    
    
    # CNN 2
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu' ))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.15))

    # CNN 3
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.15))

    #CNN 4
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.15))

    # You must flatten the data for the dense layers
    model.add(Flatten())

    #Dense 1
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.15))

    #Dense 2
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.15))
    
    # Output 
    model.add(Dense(1, activation="sigmoid"))

    optimizer = Adam(lr=0.001, decay=0.0)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model


In [85]:
def getModel2():
    
    #Build keras model
    
    model=Sequential()
    
    # Block 1
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(75, 75, 3)))
    #model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(Dropout(0.15))

    # Block 2
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
    #model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(Dropout(0.15))


    # Block 3
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
    #model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
    #model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
    #model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(Dropout(0.15))
              
    # Block 4
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
    #model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
    #model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(Dropout(0.15))
              
    # Block 5
    #model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
    #model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
    #model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
    #model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
    #model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    #
    
    
    
    
    
    
    model.add(Dropout(0.15))
    
    # Classification block
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.15))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.15))
    #model.add(Dense(1024, activation='relu'))
    #model.add(Dropout(0.15))
    #model.add(Dense(512, activation='relu'))
    #model.add(Dropout(0.15))

    # Output 
    model.add(Dense(1, activation="sigmoid"))

    optimizer = Adam(lr=0.001, decay=0.0)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model

In [86]:
model = getModel2()
model.summary()

batch_size = 20
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('.mdl_wts2.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_76 (Conv2D)           (None, 75, 75, 128)       3584      
_________________________________________________________________
max_pooling2d_54 (MaxPooling (None, 37, 37, 128)       0         
_________________________________________________________________
dropout_89 (Dropout)         (None, 37, 37, 128)       0         
_________________________________________________________________
conv2d_77 (Conv2D)           (None, 37, 37, 128)       147584    
_________________________________________________________________
max_pooling2d_55 (MaxPooling (None, 18, 18, 128)       0         
_________________________________________________________________
dropout_90 (Dropout)         (None, 18, 18, 128)       0         
_________________________________________________________________
conv2d_78 (Conv2D)           (None, 18, 18, 256)       295168    
__________

In [87]:
model.fit(Xtr_more, Ytr_more, batch_size=batch_size, epochs=50, verbose=1, callbacks=[earlyStopping, mcp_save, reduce_lr_loss], validation_split=0.25)

Train on 9929 samples, validate on 3310 samples
Epoch 1/50
9929/9929 [==============================] - 608s - loss: 0.6557 - acc: 0.5964 - val_loss: 0.6315 - val_acc: 0.7257
Epoch 2/50
9929/9929 [==============================] - 604s - loss: 0.4436 - acc: 0.7873 - val_loss: 0.4101 - val_acc: 0.8079
Epoch 3/50
9929/9929 [==============================] - 603s - loss: 0.3767 - acc: 0.8124 - val_loss: 0.6089 - val_acc: 0.7242
Epoch 4/50
9929/9929 [==============================] - 605s - loss: 0.3123 - acc: 0.8492 - val_loss: 0.3901 - val_acc: 0.8097
Epoch 5/50
9929/9929 [==============================] - 603s - loss: 0.2713 - acc: 0.8701 - val_loss: 0.2958 - val_acc: 0.8695
Epoch 6/50
9929/9929 [==============================] - 602s - loss: 0.2570 - acc: 0.8857 - val_loss: 0.2850 - val_acc: 0.8653
Epoch 7/50
9929/9929 [==============================] - 603s - loss: 0.2326 - acc: 0.8966 - val_loss: 0.2890 - val_acc: 0.8698
Epoch 8/50
9929/9929 [==============================] - 604s - 

In [90]:
model.load_weights(filepath = '.mdl_wts2.hdf5')

score = model.evaluate(Xtrain, Ytrain, verbose=1)
print('Train score:', score[0])
print('Train accuracy:', score[1])

1471/1471 [==============================] - 20s    
Train score: 0.12167462309
Train accuracy: 0.950373895309


In [91]:
df_test = pd.read_json("C:\\Users\\dshah\\Desktop\\data\\processed\\test.json")
df_test.inc_angle = df_test.inc_angle.replace('na',0)

In [92]:
Xtest = (get_scaled_imgs(df_test))
pred_test = model.predict(Xtest)

submission = pd.DataFrame({'id': df_test["id"], 'is_iceberg': pred_test.reshape((pred_test.shape[0]))})
print(submission.head(10))

submission.to_csv('submission3_ADAM(5).csv', index=False)

         id  is_iceberg
0  5941774d    0.001493
1  4023181e    1.000000
2  b20200e4    0.023205
3  e7f018bb    1.000000
4  4371c8c3    1.000000
5  a8d9b1fd    0.076170
6  29e7727e    0.069936
7  92a51ffb    1.000000
8  c769ac97    0.000014
9  aee0547d    0.000071
